In [ ]:
#%% Imports:
import matplotlib
matplotlib.use('MACOSX')

from math import sqrt
import os
import numpy as np
import mne
#import matplotlib #doesnt it import through mne already?
from copy import deepcopy

#Open data:
#sample_data_folder = mne.datasets.sample.data_path()
#kath_raw_file2 = "/Users/jenya/Documents/Oldenburg and university/Job Uni Rieger lab/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif"
#kath_raw_file = os.path.join('Katharinas_Data','sub_HT05ND16', '210811', 'mikado-1.fif')
#kath_raw_file='/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif'
kath_raw_file='./data/sub_HT05ND16/210811/mikado-1.fif/'
print(kath_raw_file)
#print(kath_raw_file2)                                   
raw = mne.io.read_raw_fif(kath_raw_file)
#raw.crop(0, 60).load_data()  # just use a fraction of data for speed here

#Print info about the data:
print(raw)
print(raw.info)
raw

#crop the data to calculate faster

raw_cropped = raw.copy()
raw_cropped.crop(0, 300) #(first 5 min)

In [ ]:
#%% Find magnetometers and gradiometers:
# unit t - magenetometer. unit M_T - gradiometer. (in this set name will end with 1 for magnet, 
# with 2and3 for grad.)

mags = []
grads = []

for i, chs in enumerate(raw_cropped.info['chs']):

    if str(chs['unit']).endswith('UNIT_T)'):
        mags.append((chs['ch_name'], i))
    elif str(chs['unit']).endswith('UNIT_T_M)'):
        grads.append((chs['ch_name'], i))

In [ ]:
#epoch channels together

events = mne.find_events(raw_cropped, stim_channel='STI101', min_duration=1.2)
epochs = mne.Epochs(raw_cropped, events, tmin=-0.2, tmax=1, preload=True, baseline = None) #, event_id=event_dict, reject=reject_criteria)
#df = epochs.to_data_frame(time_format=None, scalings=dict(mag=1, grad=1))
df = epochs.to_data_frame(time_format=None, scalings=None)

#by default, channel measurement values are scaled so that EEG data are converted 
# to µV, magnetometer data are converted to fT, and gradiometer data are converted 
# to fT/cm. These scalings can be customized through the scalings parameter, or 
# suppressed by passing scalings=dict(eeg=1, mag=1, grad=1).
df



In [ ]:
#epoch channels separately

picks_grad = mne.pick_types(raw_cropped.info, meg='grad', eeg=False, eog=False, stim=False)
picks_magn = mne.pick_types(raw_cropped.info, meg='mag', eeg=False, eog=False, stim=False)

epochs_mags = mne.Epochs(raw_cropped, events, picks=picks_magn, tmin=-0.2, tmax=1, preload=True, baseline = None)
epochs_grads = mne.Epochs(raw_cropped, events, picks=picks_grad, tmin=-0.2, tmax=1, preload=True, baseline = None)

#Present epochs as data frame - separately for mags and grads

#df_epochs_mags = epochs_mags.to_data_frame(time_format=None, scalings=dict(mag=1, grad=1))
df_epochs_mags = epochs_mags.to_data_frame(time_format=None, scalings=None)

#df_epochs_grads = epochs_grads.to_data_frame(time_format=None, scalings=dict(mag=1, grad=1))
df_epochs_grads = epochs_grads.to_data_frame(time_format=None, scalings=None)

df_epochs_mags

In [ ]:
#Compare data in 2 dfs: mags and grads together vs mags separately

#Joined df:
epoch0_ind=df.index[df['epoch'] == 0].tolist()
ch0111=df.iloc[epoch0_ind, 3+11] #all data of epoch 0 for MEG0111. 
#index of this channel in raw: 11. moved by 3 elements because data frame creates 3 additional columns
#print('data for MEG0111 of epoch 0: ', ch0111)

#Separate mags df:
epoch0_ind_mag=df_epochs_mags.index[df_epochs_mags['epoch'] == 0].tolist()
ch0111_mag=df_epochs_mags.iloc[epoch0_ind, 3] #all data of epoch 0 for MEG0111. 
#index only moved by 3 elements here only, since no extra channels in this df

#print('dfs have same data: ')
#print("A: ", ch0111[0], "\nB: ",ch0111_mag[0]) #check if the data in joined df and the magnetometer df for channel MEG0111 is same in both)

for index, value in enumerate(ch0111):
    if value != ch0111_mag[index]:
        print("Not equal:", index, "\n", value, "\n", ch0111_mag[index], "\n")

In [ ]:
#Data of f this channel and epoch from  joined DF:
print(df.iloc[epoch0_ind, 3+11])

In [ ]:
#Data of this channel and epoch from separated DF:
df_epochs_mags.iloc[epoch0_ind, 3]